In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython.display import display, clear_output


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import anndata as ad

In [2]:
import pandas as pd
import numpy as np
from collections import Counter


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from imblearn.over_sampling import SMOTE

In [3]:
rxrx19a_metadata = pd.read_csv('data/rxrx19a_metadata.csv', index_col=0)
rxrx19b_metadata = pd.read_csv('data/rxrx19b_metadata.csv', index_col=0)
rxrx19a_embeddings = pd.read_csv('data/rxrx19a_embeddings.csv', index_col=0)
rxrx19b_embeddings = pd.read_csv('data/rxrx19b_embeddings.csv', index_col=0)


rxrx19a_metadata.head()
rxrx19b_metadata.head()



,well_id,cell_type,experiment,plate,well,site,disease_condition,treatment,treatment_conc,SMILES
site_id,,,,,,,,,,
HRCE-1_1_AA02_1,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,1,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...
HRCE-1_1_AA02_2,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,2,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...
HRCE-1_1_AA02_3,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,3,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...
HRCE-1_1_AA02_4,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,4,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...
HRCE-1_1_AA03_1,HRCE-1_1_AA03,HRCE,HRCE-1,1,AA03,1,Active SARS-CoV-2,acetylcysteine,1.0,"CC(=O)N[C@H](CS)C(O)=O |a:4,r|"


,well_id,cell_type,experiment,plate,well,site,disease_condition,treatment,treatment_conc,SMILES
site_id,,,,,,,,,,
HUVEC-1_1_AA02_1,HUVEC-1_1_AA02,HUVEC,HUVEC-1,1,AA02,1,storm-severe,Ravoxertinib,1.00,CN1N=CC=C1NC1=NC=CC(=N1)C1=CC(=O)N(C=C1)C(CO)C...
HUVEC-1_1_AA03_1,HUVEC-1_1_AA03,HUVEC,HUVEC-1,1,AA03,1,storm-severe,Diphenylpyraline,0.25,"CN1CCC(CC1)OC(C1=CC=CC=C1)C1=CC=CC=C1 |c:12,14..."
HUVEC-1_1_AA04_1,HUVEC-1_1_AA04,HUVEC,HUVEC-1,1,AA04,1,storm-severe,Indirubin,1.00,O=C1NC2=CC=CC=C2\C1=C1\NC2=C(C=CC=C2)C1=O |c:5...
HUVEC-1_1_AA05_1,HUVEC-1_1_AA05,HUVEC,HUVEC-1,1,AA05,1,storm-severe,YO-01027,1.00,C[C@H](NC(=O)CC1=CC(F)=CC(F)=C1)C(=O)N[C@H]1C2...
HUVEC-1_1_AA07_1,HUVEC-1_1_AA07,HUVEC,HUVEC-1,1,AA07,1,storm-severe,taurine,1.00,NCCS(O)(=O)=O


In [4]:
rxrx19a_metadata['cell_type'].value_counts()
rxrx19a_metadata['disease_condition'].value_counts()
rxrx19a_metadata['treatment'].value_counts()

cell_type
HRCE    284080
VERO     21440
Name: count, dtype: int64

disease_condition
Active SARS-CoV-2            280376
UV Inactivated SARS-CoV-2      9120
Mock                           9120
Name: count, dtype: int64

treatment
Ribavirin                        864
Favipiravir                      864
Indomethacin                     864
Tenofovir Disoproxil Fumarate    864
Lopinavir                        864
                                ... 
fluspirilene                     144
Thiocolchicoside                 144
DEET                             144
Salmeterol                       144
Phenytoin                        144
Name: count, Length: 1669, dtype: int64

In [5]:
rxrx19b_metadata['cell_type'].value_counts()
rxrx19b_metadata['disease_condition'].value_counts()
rxrx19b_metadata['treatment'].value_counts()

cell_type
HUVEC    70384
Name: count, dtype: int64

disease_condition
storm-severe    67364
healthy          2756
Name: count, dtype: int64

treatment
Allopregnanolone    72
dorzolamide         36
trelagliptin        36
gidazepam           36
YK-4-279            36
                    ..
CPI-203             24
AZD1080             24
MK-2206             24
AMG-900             24
Lovastatin          24
Name: count, Length: 1856, dtype: int64

##We will use the B Dataset first because its smaller##

In [6]:
X = rxrx19b_embeddings.values.tolist()

X_series = pd.Series(X, index=rxrx19b_embeddings.index, name='embeddings')
X_df = X_series.to_frame()

y = rxrx19b_metadata['disease_condition']
y_df = y.to_frame()
X_df = X_df.loc[y.index]

In [7]:
X_df.head()

y_df.head()

,embeddings
site_id,
HUVEC-1_1_AA02_1,"[-0.2086878, 4.063373, -0.12942116, -1.1610049..."
HUVEC-1_1_AA03_1,"[-0.05914755, 4.103718, -1.1349981, -0.7888756..."
HUVEC-1_1_AA04_1,"[-0.15590928, 3.754905, -0.36183846, -0.206010..."
HUVEC-1_1_AA05_1,"[-0.7296535, 3.4197803, 0.059896838, -0.312398..."
HUVEC-1_1_AA07_1,"[0.08298465, 4.188676, -0.16406447, -0.0579336..."


,disease_condition
site_id,
HUVEC-1_1_AA02_1,storm-severe
HUVEC-1_1_AA03_1,storm-severe
HUVEC-1_1_AA04_1,storm-severe
HUVEC-1_1_AA05_1,storm-severe
HUVEC-1_1_AA07_1,storm-severe


In [8]:
non_nan_indices = ~y_df['disease_condition'].isna()
X_filtered = X_df[non_nan_indices]
y_filtered = y_df[non_nan_indices]

In [9]:
X = np.vstack(X_filtered['embeddings'].values)
y = y_filtered['disease_condition'].values

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

class_names = label_encoder.classes_

num_classes = len(np.unique(y_encoded))
print(f"Number of classes: {num_classes}")
print("Classes:", class_names)

Number of classes: 2
Classes: ['healthy' 'storm-severe']


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class weights:", class_weights_dict)

Class weights: {0: np.float64(12.720181405895692), 1: np.float64(0.5204579614406858)}


In [11]:
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_scaled, y_train)
print("Resampled training set shape:", Counter(y_train_resampled))

Resampled training set shape: Counter({np.int64(1): 53891, np.int64(0): 53891})


In [12]:
lr = LogisticRegression(
    class_weight='balanced', max_iter=1000, random_state=42
)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
print("\nLogistic Regression with Class Weights:")
print(classification_report(y_test, y_pred_lr, target_names=class_names))

# With SMOTE
lr_smote = LogisticRegression(max_iter=1000, random_state=42)
lr_smote.fit(X_train_resampled, y_train_resampled)
y_pred_lr_smote = lr_smote.predict(X_test_scaled)
print("Logistic Regression with SMOTE:")
print(classification_report(y_test, y_pred_lr_smote, target_names=class_names))

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)


Logistic Regression with Class Weights:
              precision    recall  f1-score   support

     healthy       0.75      0.99      0.85       551
storm-severe       1.00      0.99      0.99     13473

    accuracy                           0.99     14024
   macro avg       0.88      0.99      0.92     14024
weighted avg       0.99      0.99      0.99     14024



LogisticRegression(max_iter=1000, random_state=42)

Logistic Regression with SMOTE:
              precision    recall  f1-score   support

     healthy       0.77      0.98      0.86       551
storm-severe       1.00      0.99      0.99     13473

    accuracy                           0.99     14024
   macro avg       0.88      0.98      0.93     14024
weighted avg       0.99      0.99      0.99     14024



In [13]:
svm = SVC(
    class_weight='balanced', probability=True, random_state=42
)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
print("\nSVM with Class Weights:")
print(classification_report(y_test, y_pred_svm, target_names=class_names))

# With SMOTE
svm_smote = SVC(probability=True, random_state=42)
svm_smote.fit(X_train_resampled, y_train_resampled)
y_pred_svm_smote = svm_smote.predict(X_test_scaled)
print("SVM with SMOTE:")
print(classification_report(y_test, y_pred_svm_smote, target_names=class_names))

SVC(class_weight='balanced', probability=True, random_state=42)


SVM with Class Weights:
              precision    recall  f1-score   support

     healthy       0.84      0.98      0.91       551
storm-severe       1.00      0.99      1.00     13473

    accuracy                           0.99     14024
   macro avg       0.92      0.99      0.95     14024
weighted avg       0.99      0.99      0.99     14024



SVC(probability=True, random_state=42)

SVM with SMOTE:
              precision    recall  f1-score   support

     healthy       0.86      0.98      0.91       551
storm-severe       1.00      0.99      1.00     13473

    accuracy                           0.99     14024
   macro avg       0.93      0.98      0.95     14024
weighted avg       0.99      0.99      0.99     14024



In [14]:
rf = RandomForestClassifier(
    class_weight='balanced', n_estimators=100, random_state=42
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("\nRandom Forest with Class Weights:")
print(classification_report(y_test, y_pred_rf, target_names=class_names))

# With SMOTE
#rf_smote = RandomForestClassifier(n_estimators=100, random_state=42)
#rf_smote.fit(X_train_resampled, y_train_resampled)
#y_pred_rf_smote = rf_smote.predict(X_test)
#print("Random Forest with SMOTE:")
#print(classification_report(y_test, y_pred_rf_smote, target_names=class_names))

RandomForestClassifier(class_weight='balanced', random_state=42)


Random Forest with Class Weights:
              precision    recall  f1-score   support

     healthy       0.85      0.78      0.82       551
storm-severe       0.99      0.99      0.99     13473

    accuracy                           0.99     14024
   macro avg       0.92      0.89      0.90     14024
weighted avg       0.99      0.99      0.99     14024



RandomForestClassifier(random_state=42)

Random Forest with SMOTE:
              precision    recall  f1-score   support

     healthy       0.00      0.00      0.00       551
storm-severe       0.96      1.00      0.98     13473

    accuracy                           0.96     14024
   macro avg       0.48      0.50      0.49     14024
weighted avg       0.92      0.96      0.94     14024



/opt/miniconda/envs/proj/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda/envs/proj/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda/envs/proj/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'multi:softprob',
    'num_class': len(class_names),
    'eval_metric': 'mlogloss',
    'seed': 42,
    'lambda': 1,
}


scale_pos_weight = class_weights
params['scale_pos_weight'] = scale_pos_weight

# Train XGBoost model
xgb_model = xgb.train(params, dtrain, num_boost_round=100)
y_pred_xgb_probs = xgb_model.predict(dtest)
y_pred_xgb = np.argmax(y_pred_xgb_probs, axis=1)
print("\nXGBoost with Class Weights:")
print(classification_report(y_test, y_pred_xgb, target_names=class_names))

/opt/miniconda/envs/proj/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [07:39:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)



XGBoost with Class Weights:
              precision    recall  f1-score   support

     healthy       0.88      0.91      0.90       551
storm-severe       1.00      1.00      1.00     13473

    accuracy                           0.99     14024
   macro avg       0.94      0.95      0.95     14024
weighted avg       0.99      0.99      0.99     14024

